In [50]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from tensorflow.keras import Sequential, Model, Input
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
import os
import keras

In [9]:
cfd_df_raw = pd.read_csv("CFD/metadata.csv")
cfd_df_raw.head(10)

,Model,EthnicitySelf,GenderSelf,AgeSelf,AgeRated,FemaleProb,MaleProb,AsianProb,BlackProb,LatinoProb,...,UpperHeadLength,MidfaceLength,ChinLength,ForeheadHeight,CheekboneHeight,CheekboneProminence,FaceRoundness,fWHR1,fWHR2,RaterN
0,AF-200,A,F,NaN,32.571429,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.414099,0.326797,0.130719,0.264706,0.388189,91.5,0.545752,1.921146,NaN,28
1,AF-201,A,F,NaN,23.666667,1.000000,0.000000,0.962963,0.000000,0.000000,...,0.414414,0.329279,0.144595,0.300901,0.383784,146.0,0.488288,1.901129,NaN,27
2,AF-202,A,F,NaN,24.448276,0.827586,0.172414,0.310345,0.068966,0.137931,...,0.411080,0.310317,0.173424,0.298475,0.397029,58.0,0.481333,1.888249,NaN,29
3,AF-203,A,F,NaN,22.758621,1.000000,0.000000,0.758621,0.000000,0.068966,...,0.354407,0.343793,0.169820,0.272266,0.421089,87.5,0.500231,1.863719,NaN,29
4,AF-204,A,F,NaN,30.137931,1.000000,0.000000,0.827586,0.000000,0.068966,...,0.438931,0.293045,0.180237,0.293893,0.371925,73.5,0.513571,1.935783,NaN,29
5,AF-205,A,F,NaN,26.592593,1.000000,0.000000,0.846154,0.000000,0.000000,...,0.408727,0.352182,0.158949,0.270481,0.408281,69.5,0.516919,1.718556,NaN,26
6,AF-206,A,F,NaN,26.523810,0.857143,0.142857,1.000000,0.000000,0.000000,...,0.383958,0.360228,0.172283,0.253916,0.399146,109.5,0.533935,1.847015,NaN,20
7,AF-207,A,F,NaN,28.413793,1.000000,0.000000,0.035714,0.428571,0.035714,...,0.437947,0.273875,0.197728,0.329407,0.386201,3.0,0.553218,1.954528,NaN,28
8,AF-208,A,F,NaN,28.538462,1.000000,0.000000,0.230769,0.115385,0.384615,...,0.428939,0.295835,0.214684,0.288751,0.382138,50.0,0.551739,2.053120,NaN,26
9,AF-209,A,F,NaN,22.560000,1.000000,0.000000,0.080000,0.080000,0.400000,...,0.385501,0.326652,0.188913,0.267164,0.410235,122.5,0.455437,1.801707,NaN,25


In [36]:
def getFileNames(target):
    files = []
    file_count = 0
    path = "CFD/Images/CFD/%s/" % (target)
    
    for r, d, f in os.walk(path):
        for file in f:
            if ('.jpg' in file) or ('.jpeg' in file) or ('.png' in file):
                files.append(file)
    return files
 
cfd_df_raw["files"] = cfd_df_raw.Model.apply(getFileNames)

In [37]:
cfd_instances = []
for index, instance in cfd_df_raw.iterrows():
    folder = instance.Model
    score = instance['Attractive']
    for file in instance.files:
        tmp_instance = []
        tmp_instance.append(folder)
        tmp_instance.append(file)
        tmp_instance.append(score)
        cfd_instances.append(tmp_instance)
 
df = pd.DataFrame(cfd_instances, columns = ["folder", "file", "score"])

In [40]:
def retrievePixels(path):
    img = load_img(path, grayscale=False, target_size=(224, 224))
    x = img_to_array(img).reshape(1, -1)[0]
    return x
 
df['exact_file'] = "CFD/Images/CFD/"+df["folder"]+"/"+df['file']
df['pixels'] = df['exact_file'].apply(retrievePixels)


In [41]:
def findEmotion(file):
    #sample file name CFD-WM-040-023-HO.jpg
    file_name = file.split(".")[0] #[1] is jpg
    emotion = file_name.split("-")[4]
    return emotion
 
df['emotion'] = df.file.apply(findEmotion)
 
#include neutral, happen open mouth and happy close mouth
df = df[(df.emotion == 'N') | (df.emotion == 'HO') | (df.emotion == 'HC')]

In [42]:
features = []
pixels = df['pixels'].values
for i in range(0, pixels.shape[0]):
    features.append(pixels[i])
 
features = np.array(features)
features = features.reshape(features.shape[0], 224, 224, 3)
 
features = features / 255

In [45]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(features, df.score.values, test_size=0.3, random_state=17)

In [47]:
base_model = Sequential()
base_model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
base_model.add(Conv2D(64, (3, 3), activation='relu'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(64, (3, 3), activation='relu'))
base_model.add(MaxPooling2D((2,2), strides=(2,2)))
 
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(128, (3, 3), activation='relu'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(128, (3, 3), activation='relu'))
base_model.add(MaxPooling2D((2,2), strides=(2,2)))
 
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(256, (3, 3), activation='relu'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(256, (3, 3), activation='relu'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(256, (3, 3), activation='relu'))
base_model.add(MaxPooling2D((2,2), strides=(2,2)))
 
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(512, (3, 3), activation='relu'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(512, (3, 3), activation='relu'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(512, (3, 3), activation='relu'))
base_model.add(MaxPooling2D((2,2), strides=(2,2)))

base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(512, (3, 3), activation='relu'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(512, (3, 3), activation='relu'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Conv2D(512, (3, 3), activation='relu'))
base_model.add(MaxPooling2D((2,2), strides=(2,2)))
 
base_model.add(Conv2D(4096, (7, 7), activation='relu'))
base_model.add(Dropout(0.5))
base_model.add(Conv2D(4096, (1, 1), activation='relu'))
base_model.add(Dropout(0.5))
base_model.add(Conv2D(2622, (1, 1)))
base_model.add(Flatten())
base_model.add(Activation('softmax'))
 
#pre-trained weights of vgg-face model.
#you can find it here: https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
#related blog post: https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/
base_model.load_weights('vgg_face_weights.h5')

In [48]:
num_of_classes = 1 #this is a regression problem
 
#freeze all layers of VGG-Face except last 7 one
for layer in base_model.layers[:-7]:
    layer.trainable = False
 
base_model_output = Sequential()
base_model_output = Flatten()(base_model.layers[-4].output)
base_model_output = Dense(num_of_classes)(base_model_output)
 
attractiveness_model = Model(inputs=base_model.input, outputs=base_model_output)


In [ ]:
import tensorflow as tf
attractiveness_model.compile(loss='mean_squared_error'
, optimizer=keras.optimizers.Adam())
 
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='attractiveness.hdf5'
    , monitor = "val_loss", verbose=1
, save_best_only=True, mode = 'auto'
)
 
earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
 
score = attractiveness_model.fit(train_x, train_y, epochs=5000
    , validation_data=(val_x, val_y), callbacks=[checkpointer, earlyStop]
)


Epoch 1/5000
20/20 [==============================] - ETA: 0s - loss: 4.8863 
Epoch 00001: val_loss improved from inf to 0.67029, saving model to attractiveness.hdf5
20/20 [==============================] - 386s 19s/step - loss: 4.8863 - val_loss: 0.6703
Epoch 2/5000
20/20 [==============================] - ETA: 0s - loss: 0.7281 
Epoch 00002: val_loss improved from 0.67029 to 0.45054, saving model to attractiveness.hdf5
20/20 [==============================] - 429s 21s/step - loss: 0.7281 - val_loss: 0.4505
Epoch 3/5000
20/20 [==============================] - ETA: 0s - loss: 0.4687 
Epoch 00003: val_loss improved from 0.45054 to 0.34004, saving model to attractiveness.hdf5
20/20 [==============================] - 407s 20s/step - loss: 0.4687 - val_loss: 0.3400
Epoch 4/5000
20/20 [==============================] - ETA: 0s - loss: 0.4165 
Epoch 00004: val_loss did not improve from 0.34004
20/20 [==============================] - 466s 23s/step - loss: 0.4165 - val_loss: 0.3481
Epoch 5/5